## Imports

In [1]:
import os
import math

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 

import flwr as fl
import tensorflow as tf

from keras.models import Sequential, load_model #, clone_model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D

In [2]:
from typing import List, Tuple, cast, Union, Optional, Dict

from flwr.common import (
    Parameters,
    Scalar,
)

XY = Tuple[np.ndarray, np.ndarray]
XYList = List[XY]
PartitionedDataset = List[Tuple[XY, XY]]

In [3]:
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

tf.compat.v1.disable_eager_execution()

# Number of clients
NUM_CLIENTS = 1000

## Load model

In [4]:
reference_model = load_model("model.h5")

## Dataset partition

In [5]:
df = pd.read_pickle("./data/dataset.pkl")

# Add one dimension for the channel
X = np.array(df['features'].tolist())
X = X.reshape((*X.shape, 1))
y = np.array(df['labels_categorical'].tolist())

# As there is unbalance for some classes I am going to stratify it so we have the same proportion in train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.30, 
                                                    random_state=1, 
                                                    stratify=y)
# Create validation and test
X_test, X_val, Y_test, Y_val = train_test_split(X_test, 
                                                Y_test, 
                                                test_size=0.5, 
                                                random_state=1, 
                                                stratify=Y_test)

print(X_train.shape, X_val.shape, X_test.shape)

(6112, 39, 174, 1) (1310, 39, 174, 1) (1310, 39, 174, 1)


## Helper functions

In [6]:
def create_model(num_classes=10, input_shape=None, dropout_ratio=None):
    model = Sequential()
    if input_shape is None:
        model.add(Input(shape=(None, None, 1)))
    else:
        model.add(Input(shape=input_shape))
    model.add(Conv2D(filters=16, kernel_size=(2, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 3)))
    model.add(Conv2D(filters=32, kernel_size=(2, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64, kernel_size=(2, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=128, kernel_size=(2, 4), activation='relu'))
    model.add(GlobalAveragePooling2D())
    if dropout_ratio is not None:
        model.add(Dropout(dropout_ratio))
    # Add dense linear layer
    model.add(Dense(num_classes, activation='softmax'))
    return model

## FL client definition

In [7]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=1, verbose=2)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.x_val, self.y_val, verbose=2)
        return loss, len(self.x_val), {"accuracy": acc}


def client_fn(cid: str) -> fl.client.Client:
    global X_train, Y_train

    # Create model
    model = create_model(input_shape=X_train.shape[1:], dropout_ratio=0.5)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

    # Load data partition (divide MNIST into NUM_CLIENTS distinct partitions)
    x_train = X_train.copy()
    y_train = Y_train.copy()
    partition_size = math.floor(len(x_train) / NUM_CLIENTS)
    idx_from, idx_to = int(cid) * partition_size, (int(cid) + 1) * partition_size
    x_train_cid = x_train[idx_from:idx_to] / 255.0
    y_train_cid = y_train[idx_from:idx_to]

    # Use 10% of the client's training data for validation
    split_idx = math.floor(len(x_train) * 0.9)
    x_train_cid, y_train_cid = x_train_cid[:split_idx], y_train_cid[:split_idx]
    x_val_cid, y_val_cid = x_train_cid[split_idx:], y_train_cid[split_idx:]

    # Create and return client
    return FlowerClient(model, x_train_cid, y_train_cid, x_val_cid, y_val_cid)

## FL server definition

In [8]:
class SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]],
        failures: List[Union[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:

        # Call aggregate_fit from base class (FedAvg) to aggregate parameters and metrics
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)

        if aggregated_parameters is not None:
            # Convert `Parameters` to `List[np.ndarray]`
            aggregated_ndarrays: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_parameters)

            # Save aggregated_ndarrays
            print(f"Saving round {server_round} aggregated_ndarrays...")
            np.savez(f"./weights/round-{server_round}-weights.npz", *aggregated_ndarrays)

        return aggregated_parameters, aggregated_metrics

In [9]:
# Create FedAvg strategy
strategy=SaveModelStrategy(
    fraction_fit=0.2,  # Sample 10% of available clients for training
    fraction_evaluate=0.05,  # Sample 5% of available clients for evaluation
    min_fit_clients=200,  # Never sample less than 10 clients for training
    min_evaluate_clients=100,  # Never sample less than 5 clients for evaluation
    min_available_clients=int(NUM_CLIENTS * 0.75),  # Wait until at least 75 clients are available
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=20),
    strategy=strategy,
)

INFO flower 2022-09-15 23:59:38,204 | app.py:142 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
INFO flower 2022-09-15 23:59:53,447 | app.py:176 | Flower VCE: Ray initialized with resources: {'memory': 63597224960.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 31541667840.0, 'CPU': 16.0, 'GPU': 1.0}
INFO flower 2022-09-15 23:59:53,450 | server.py:86 | Initializing global parameters
INFO flower 2022-09-15 23:59:53,450 | server.py:270 | Requesting initial parameters from one random client
INFO flower 2022-09-15 23:59:56,365 | server.py:274 | Received initial parameters from one random client
INFO flower 2022-09-15 23:59:56,366 | server.py:88 | Evaluating initial parameters
INFO flower 2022-09-15 23:59:56,366 | server.py:101 | FL starting
DEBUG flower 2022-09-15 23:59:56,367 | server.py:215 | fit_round 1: strategy sampled 200 clients (out of 1000)


 pid=142204) 1/1 - 1s - loss: 2.2972 - accuracy: 0.1667 - 1s/epoch - 1s/step
 pid=142204) 1/1 - 1s - loss: 2.2979 - accuracy: 0.1667 - 1s/epoch - 1s/step
 pid=142204) 1/1 - 1s - loss: 2.3041 - accuracy: 0.1667 - 995ms/epoch - 995ms/step
 pid=142204) 1/1 - 1s - loss: 2.3049 - accuracy: 0.0000e+00 - 1s/epoch - 1s/step
 pid=147144) 1/1 - 2s - loss: 2.3041 - accuracy: 0.1667 - 2s/epoch - 2s/step
 pid=130848) 1/1 - 2s - loss: 2.3028 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step


 pid=142204) WARNING:tensorflow:5 out of the last 5 calls to <function Model.make_train_function.<locals>.train_function at 0x000001D54AB76EE0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


 pid=142204) 1/1 - 1s - loss: 2.2896 - accuracy: 0.3333 - 1s/epoch - 1s/step
 pid=138528) 1/1 - 2s - loss: 2.2955 - accuracy: 0.1667 - 2s/epoch - 2s/step
 pid=142472) 1/1 - 2s - loss: 2.2960 - accuracy: 0.1667 - 2s/epoch - 2s/step
 pid=146136) 1/1 - 2s - loss: 2.3021 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=137644) 1/1 - 2s - loss: 2.2954 - accuracy: 0.3333 - 2s/epoch - 2s/step
 pid=139920) 1/1 - 2s - loss: 2.3062 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=147460) 1/1 - 2s - loss: 2.3022 - accuracy: 0.1667 - 2s/epoch - 2s/step
 pid=104964) 1/1 - 2s - loss: 2.2983 - accuracy: 0.1667 - 2s/epoch - 2s/step
 pid=142424) 1/1 - 2s - loss: 2.3045 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=141916) 1/1 - 2s - loss: 2.3115 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=132740) 1/1 - 2s - loss: 2.2999 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=143560) 1/1 - 2s - loss: 2.3010 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=136928) 1/1 - 2s - loss: 2.2910 - accuracy: 0.3

 pid=142204) WARNING:tensorflow:6 out of the last 6 calls to <function Model.make_train_function.<locals>.train_function at 0x000001D54AD4BF70> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


...

 pid=104964) 1/1 - 1s - loss: 2.3046 - accuracy: 0.0000e+00 - 1s/epoch - 1s/step
 pid=139920) 1/1 - 1s - loss: 2.2986 - accuracy: 0.3333 - 1s/epoch - 1s/step
 pid=142424) 1/1 - 1s - loss: 2.2921 - accuracy: 0.1667 - 1s/epoch - 1s/step
 pid=132740) 1/1 - 1s - loss: 2.2742 - accuracy: 0.3333 - 960ms/epoch - 960ms/step
 pid=136928) 1/1 - 1s - loss: 2.2870 - accuracy: 0.1667 - 948ms/epoch - 948ms/step
 pid=138528) 1/1 - 1s - loss: 2.2952 - accuracy: 0.1667 - 899ms/epoch - 899ms/step
 pid=143560) 1/1 - 1s - loss: 2.3172 - accuracy: 0.0000e+00 - 932ms/epoch - 932ms/step


DEBUG flower 2022-09-16 00:03:54,449 | server.py:165 | evaluate_round 7: strategy sampled 100 clients (out of 1000)


Saving round 7 aggregated_ndarrays...


DEBUG flower 2022-09-16 00:03:59,665 | server.py:179 | evaluate_round 7 received 0 results and 100 failures
DEBUG flower 2022-09-16 00:03:59,666 | server.py:215 | fit_round 8: strategy sampled 200 clients (out of 1000)


 pid=132740) 1/1 - 1s - loss: 2.2992 - accuracy: 0.0000e+00 - 1s/epoch - 1s/step
 pid=138528) 1/1 - 2s - loss: 2.2912 - accuracy: 0.3333 - 2s/epoch - 2s/step
 pid=139920) 1/1 - 1s - loss: 2.2987 - accuracy: 0.1667 - 1s/epoch - 1s/step
 pid=142424) 1/1 - 2s - loss: 2.2977 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=141916) 1/1 - 2s - loss: 2.2963 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=143560) 1/1 - 2s - loss: 2.3120 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=136928) 1/1 - 2s - loss: 2.2758 - accuracy: 0.5000 - 2s/epoch - 2s/step
 pid=142204) 1/1 - 2s - loss: 2.2953 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=146136) 1/1 - 2s - loss: 2.2980 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=104964) 1/1 - 2s - loss: 2.2938 - accuracy: 0.1667 - 2s/epoch - 2s/step
 pid=147144) 1/1 - 2s - loss: 2.2938 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
 pid=130848) 1/1 - 2s - loss: 2.2852 - accuracy: 0.1667 - 2s/epoch - 2s/step
 pid=147460) 1/1 - 2s - loss: 2.3085 - accuracy: